In [1]:
# general packages
import random
import warnings
import seaborn as sns


from dask import dataframe as dd
from dask.distributed import Client

warnings.simplefilter('ignore')
sns.set_style('whitegrid')

In [2]:
def readHugeCsvFileAsDataFrame(file_path):
    dask_df = dd.read_csv(file_path, low_memory=False, blocksize=50000)  # 50MB chunk-size
    return dask_df.compute()

In [3]:
path_to_file = 'D:\\ML_Sessional\\train.csv'
scheduler = Client()
df = scheduler.submit(readHugeCsvFileAsDataFrame, path_to_file)
workingDataFrame = df.result()

In [4]:
from sklearn.utils import shuffle
workingDataFrame = shuffle(workingDataFrame)
workingDataFrame.reset_index(inplace=True, drop=True)

adding a new column `kfold` and giving all value as `-1`

In [5]:
workingDataFrame.loc[:,'kfold'] = -1
workingDataFrame

,image_id,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme,kfold
0,Train_6579,144,0,0,ষ্প,-1
1,Train_138692,56,0,2,র্ড,-1
2,Train_170764,96,1,1,পাঁ,-1
3,Train_45514,52,0,0,ঞ্জ,-1
4,Train_144449,22,7,0,খে,-1
...,...,...,...,...,...,...
200835,Train_197369,53,2,5,ট্রি,-1
200836,Train_51200,25,0,0,গ্ন,-1
200837,Train_5969,79,2,4,ধ্যি,-1
200838,Train_154707,89,9,0,ন্দো,-1


In [6]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

x = workingDataFrame.image_id.values
y = workingDataFrame[['grapheme_root','vowel_diacritic','consonant_diacritic']].values

mskf = MultilabelStratifiedKFold(n_splits=5)

for fold, (train,validation) in enumerate(mskf.split(x,y)):
    # print("train : ",train," Validation : ",validation)
    workingDataFrame.loc[validation,"kfold"] = fold

# print(workingDataFrame.kfold.value_counts())
workingDataFrame.to_csv('../input/train_folds.csv',index=False)